In [9]:
FileLength = 6000/0.256 #Blocks
SizeOfBlock = 0.256 #kb

print(FileLength)

FATPtr = 4 #Bytes

#(Direct, Single Indirect, Double Indirect, Triply Indirect, Quad indirect if applicable)
Direct = (8,1,1,1,0) #Entries
EntriesPerSubLayer = 64 #Entries
SizeOfIndexBlock = 4 #Bytes


BlockToRead = 1099 #Block


TotalSizeKB = SizeOfBlock*FileLength
print("Size of File:\n   Bytes:", TotalSizeKB*1024, "\n   KB:", TotalSizeKB, "\n   MB:", (TotalSizeKB/1024))


print("\n----------------\n")


print("FAT:")
FAT(FileLength, SizeOfBlock, FATPtr, BlockToRead)


print("\n----------------\n")


print("Unix:")
Unix(FileLength, SizeOfBlock, Direct, EntriesPerSubLayer, BlockToRead)

23437.5
Size of File:
   Bytes: 6144000.0 
   KB: 6000.0 
   MB: 5.859375

----------------

FAT:
Number Of Bytes To Store The File:
   Bytes: 93750.0 
   KB: 91.552734375
Number of Operations to Read the 1099 th block:
    1098

----------------

Unix:
Blocks in Each Layer:
   Direct: 8 
   Single Indirect: 64 
   Double Indirect: 4096 
   Triple Indirect: 19269.5 
   Quad Indirect: 0
Number of Index Blocks in Each Layer:
   Direct: 1 
   Single Indirect: 1 
   Double Indirect: 65 
   Triple Indirect: 308 
   Quad Indirect: 0 
   Total: 375
Number Of Bytes To Store The File:
   Bytes: 6144.0 
   KB: 6.0
Number of Operations to Read the 1099 th block:
    5


In [3]:
def FAT(FileLength, SizeOfBlock, FATPtr, blockToRead):
    print("Number Of Bytes To Store The File:\n   Bytes:", FATPtr * FileLength, "\n   KB:", (FATPtr * FileLength)/1024)
    print("Number of Operations to Read the", blockToRead, "th block:\n   ", blockToRead-1)

In [4]:
def Unix(FileLength, SizeOfBlock, Direct, EntriesPerSubLayer, blockToRead):
    FileLengthCopy = FileLength
    topLayer = Direct[0]
    FirstIndirect = EntriesPerSubLayer*Direct[1]
    SecondIndirect = EntriesPerSubLayer*EntriesPerSubLayer*Direct[2]
    ThirdIndirect = EntriesPerSubLayer*EntriesPerSubLayer*EntriesPerSubLayer*Direct[3]
    FourthIndirect = EntriesPerSubLayer*EntriesPerSubLayer*EntriesPerSubLayer*EntriesPerSubLayer*Direct[4]
    
    storage = [0 for x in range(5)]
    
    if FileLengthCopy-topLayer > 0:
        FileLengthCopy -= topLayer
        storage[0] += topLayer
    else:
        storage[0] = FileLengthCopy
        FileLengthCopy = 0
        
    if FileLengthCopy-FirstIndirect > 0:
        FileLengthCopy -= FirstIndirect
        storage[1] += FirstIndirect
    else:
        storage[1] = FileLengthCopy
        FileLengthCopy = 0
        
    if FileLengthCopy-SecondIndirect > 0:
        FileLengthCopy -= SecondIndirect
        storage[2] += SecondIndirect
    else:
        storage[2] = FileLengthCopy
        FileLengthCopy = 0
        
    if FileLengthCopy-ThirdIndirect > 0:
        FileLengthCopy -= ThirdIndirect
        storage[3] += ThirdIndirect
    else:
        storage[3] = FileLengthCopy
        FileLengthCopy = 0
        
    if FileLengthCopy-FourthIndirect > 0:
        FileLengthCopy -= FourthIndirect
        storage[4] += FourthIndirect
    else:
        storage[4] = FileLengthCopy
        FileLengthCopy = 0
    
    storageCopy = storage.copy()
    output1 = [0,0,0,0,0]

    temp = 0
    total = 0
    #Layer 0
    while storage[0] > 0:
        storage[0] -= Direct[0]
        temp += 1
    total += temp
    output1[0] = total
    
    #Layer 1
    temp = 0
    total = 0
    while storage[1] > 0:
        storage[1] -= EntriesPerSubLayer
        temp += 1
    total += temp
    output1[1] = total
    
    #Layer 2
    temp = 0
    total = 0
    while storage[2] > 0:
        storage[2] -= EntriesPerSubLayer
        temp += 1
    total += temp
    temp2 = 0
    while temp > 0:
        temp -= EntriesPerSubLayer
        temp2 += 1
    total += temp2
    output1[2] = total
    
    #Layer 3
    temp = 0
    total = 0
    while storage[3] > 0:
        storage[3] -= EntriesPerSubLayer
        temp += 1
    total += temp
    temp2 = 0
    while temp > 0:
        temp -= EntriesPerSubLayer
        temp2 += 1
    total += temp2
    temp3 = 0
    while temp2 > 0:
        temp2 -= EntriesPerSubLayer
        temp3 += 1
    total += temp3
    output1[3] = total
    
    #Layer 4
    temp = 0
    total = 0
    while storage[4] > 0:
        storage[4] -= EntriesPerSubLayer
        temp += 1
    total += temp
    temp2 = 0
    while temp > 0:
        temp -= EntriesPerSubLayer
        temp2 += 1
    total += temp2
    temp3 = 0
    while temp2 > 0:
        temp2 -= EntriesPerSubLayer
        temp3 += 1
    total += temp3
    temp4 = 0
    while temp3 > 0:
        temp3 -= EntriesPerSubLayer
        temp4 += 1
    total += temp4
    output1[4] = total
    
    print("Blocks in Each Layer:\n   Direct:", storageCopy[0], "\n   Single Indirect:", storageCopy[1], "\n   Double Indirect:", storageCopy[2], "\n   Triple Indirect:", storageCopy[3], "\n   Quad Indirect:", storageCopy[4])
    
    print("Number of Index Blocks in Each Layer:\n   Direct:", output1[0], "\n   Single Indirect:", output1[1], "\n   Double Indirect:", output1[2], "\n   Triple Indirect:", output1[3], "\n   Quad Indirect:", output1[4], "\n   Total:", sum(output1))
    print("Number Of Bytes To Store The File:\n   Bytes:", EntriesPerSubLayer * SizeOfBlock * sum(output1), "\n   KB:", (EntriesPerSubLayer * SizeOfBlock * sum(output1))/1024)
    
    output2 = 0
    
    #Last bit:
    if (blockToRead < topLayer):
        output2 = 1
    elif (blockToRead < topLayer + FirstIndirect):
        output2 = 3
    elif (blockToRead < topLayer + FirstIndirect + SecondIndirect):
        output2 = 5
    elif (blockToRead < topLayer + FirstIndirect + SecondIndirect + ThirdIndirect):
        output2 = 7
    elif (blockToRead < topLayer + FirstIndirect + SecondIndirect + ThirdIndirect + FourthIndirect):
        output2 = 9
    print("Number of Operations to Read the", blockToRead, "th block:\n   ", output2)